In [1]:
import numpy as np

In [2]:
data_properties = {
    'n_obs': 10000, 
    'beta': np.array([0.25, 0.75])
}

In [3]:
def gen_data(dgp):

    np.random.seed(1234567890)

    # unpack dgp 
    n_obs = dgp['n_obs']
    beta  = dgp['beta']

    # Create X's 
    iota = np.ones(n_obs).reshape(n_obs,1)
    X    = np.random.random(size = [n_obs, 1])

    X = np.column_stack(tup = (iota, X))

    # Create epsilon
    epsilon = np.random.gumbel(size = [n_obs, 2])

    # Create utilities
    u0 = epsilon[:,0]
    u1 = X @ beta + epsilon[:, 1]

    u = np.column_stack(tup = (u0, u1))

    choice = np.argmax(u, axis = 1).reshape(n_obs, 1)

    return choice, X

In [4]:
y, X = gen_data(data_properties)

In [5]:
y

array([[1],
       [0],
       [1],
       ...,
       [1],
       [1],
       [1]])

In [6]:
X

array([[1.        , 0.61879477],
       [1.        , 0.59162363],
       [1.        , 0.88868359],
       ...,
       [1.        , 0.58973078],
       [1.        , 0.14467233],
       [1.        , 0.72604559]])

In [7]:
from scipy.stats import logistic

In [8]:
def choice_prob(X, beta_guess):

    choice_pr = logistic.cdf(X @ beta_guess)

    return choice_pr

In [9]:
def log_like_i(beta_guess, y, X):

    beta_guess = beta_guess.reshape(X.shape[1] , 1)

    choice_pr = choice_prob(X, beta_guess)

    ll_i = np.log( (y == 1) * choice_pr + (y == 0) * (1 - choice_pr) )

    return ll_i

In [10]:
# my_guess = np.array([1,1]).reshape(2,1)
# value = log_like_i(beta_guess = np.array([1,1]), y= y , X = X)

In [11]:
def log_like(beta_guess, y, X):

    beta_guess = beta_guess.reshape(X.shape[1] , 1) 

    ll_i = log_like_i(beta_guess, y, X)

    return - ll_i.sum()

In [12]:
log_like(beta_guess = np.array([1,2]), y= y , X = X)

7981.456841937576

In [13]:
from scipy.optimize import minimize

In [32]:
my_start = np.array([0.5, 0.5])

res = minimize(log_like, 
         my_start, 
         args = (y, X), 
         method = 'SLSQP', # BFGS or L-BFGS-B without providing jacobian         tol = 1e-12
)

In [33]:
print(res)

     fun: 6365.154246768508
     jac: array([-0.00543213, -0.00372314])
 message: 'Optimization terminated successfully.'
    nfev: 26
     nit: 5
    njev: 5
  status: 0
 success: True
       x: array([0.32801964, 0.68705033])


In [16]:
import scipy 

scipy.__version__

'1.4.1'

In [17]:
beta_hat = res.x
print(beta_hat)

[0.3280191  0.68705659]


In [18]:
np.sqrt(np.diag(res.hess_inv))

array([0.02426733, 0.02024752])